### One-step forecasts

In this notebook we will explore creating a modified version of make_evaluation_predictions that 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions, make_evaluation_predictions_onestep
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.trainer import Trainer

import matplotlib.pyplot as plt

In [ ]:
# we pick m4_hourly as it only contains a few hundred time series
dataset = get_dataset("m4_hourly", regenerate=False)

In [ ]:
estimator = SimpleFeedForwardEstimator(
    prediction_length=dataset.metadata.prediction_length,
    freq=dataset.metadata.freq,
    trainer=Trainer(epochs=5, num_batches_per_epoch=10),
)

predictor = estimator.train(dataset.train)

forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_samples=100
)

In [ ]:
estimator_onestep = SimpleFeedForwardEstimator(
    prediction_length=1,
    freq=dataset.metadata.freq,
    trainer=Trainer(epochs=5, num_batches_per_epoch=10),
)

predictor_onestep = estimator_onestep.train(dataset.train)

forecast_onestep_it, ts_onestep_it = make_evaluation_predictions_onestep(
    dataset.test, predictor=predictor_onestep, num_samples=100, prediction_length=dataset.metadata.prediction_length
)

In [ ]:
def plot_prob_forecasts(ts_entry, forecast_entry):
    plot_length = 150
    prediction_intervals = (50.0, 90.0)
    legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    fig, ax = plt.subplots(1, 1, figsize=(10, 7))
    ts_entry[-plot_length:].plot(ax=ax)  # plot the time series
    forecast_entry.plot(prediction_intervals=prediction_intervals, color='g')
    plt.grid(which="both")
    plt.legend(legend, loc="upper left")
    plt.show()

In [ ]:
ts_entry = next(iter(ts_it))
forecast_entry = next(iter(forecast_it))

In [ ]:
plot_prob_forecasts(ts_entry, forecast_entry)

In [ ]:
ts_onestep_entry = next(iter(ts_onestep_it))
forecast_onestep_entry = next(iter(forecast_onestep_it))

In [ ]:
plot_prob_forecasts(ts_onestep_entry, forecast_onestep_entry)